In [1]:
ckpt_path = "logs/1730961830/weights/epoch10_loss1.7540_metric0.3584.pt"

In [ ]:
for i in range(8, 12):
    novel_model = AdapterSegGPT(base_model)
    ckpt = T.load(osp.join(args.ckpt_path, f'{i}_test.pt'), map_location='cpu')
    novel_model.image_tensor.data = ckpt['model_state_dict']['image_tensor']
    novel_model.mask_tensor.data = ckpt['model_state_dict']['mask_tensor']
    print(f'Adapter for class {i} loaded')

    novel_model = novel_model.to(args.device)
    novel_model.eval()

    print(f'Running inference for novel class {i}')
    for file in tqdm(os.listdir(args.dataset_dir)):
        outdir = osp.join(args.outdir, f'novel_{i}')
        NovelInference.inference_image_with_crop(novel_model, args.device, os.path.join(args.dataset_dir, file), i, outdir=outdir, split=split_config[i])
        tgt_path = os.path.join(outdir, 'color', file.replace('.tif', '.png'))
        lbl_path = os.path.join(outdir, 'label', file.replace('.tif', '.png'))
        if split_config[i] == 2:
            NovelInference.inference_stitch(novel_model, 'cuda', os.path.join(args.dataset_dir, file), i, tgt_path, lbl_path, outdir=outdir, split=2, width=4)